In [3]:
#!/usr/bin/env python3
"""
input:
    - `{options.black_data_folder}/cluster_is_black_ground_truth.zarr` ground truth clusters from `ub_ground_truth.py`
    - `{DIR_PARSED}/{options.currency}/heur_{options.heuristic}_data/` clustering data
    - `{DIR_PARSED}/{options.currency}.cfg` blockchain data
outputs:
    * zarr file: `cluster_is_black_when_block.zarr` index is cluster id, value is int block when the cluster became black which can also represent time.
"""
# here in this script we replicate the diffusion and from ground-truth we see how users turn black block by block

import blocksci

import sys, os, os.path, socket
import numpy as np
import networkx as nx
import zarr
import time
from tqdm import tqdm
import pandas as pd
from csv import DictWriter, DictReader
import pickle as pkl
from datetime import datetime, timedelta
from itertools import compress
from scipy.sparse import random

from util import SYMBOLS, DIR_BCHAIN, DIR_PARSED, SimpleChrono

# Sparse matrix example:

In [15]:
np.random.seed(10)
matrix = random(591692716, 591692716, format='dok', density=0.4)
matrix[1, 1] = 33
matrix[(2, 1)] = 10
print(matrix.toarray())
print(dict(matrix))

[[ 0.          0.          0.16911084]
 [ 0.         33.          0.        ]
 [ 0.19806286 10.          0.22479665]]
{(2, 0): 0.19806286475962398, (2, 1): 10.0, (0, 2): 0.16911083656253545, (2, 2): 0.22479664553084766, (1, 1): 33.0}


# Gray Scale Model

### Important Variables:

In this task we will be implementing the SIR model with the following system parameters:

*   $a_{i}(b)$: total amount of assets hold by user i by the end of block b.
*   $d_{i}(b)$: total amount of black assets hold by node i by the end of block b.
*   $x_{i}(b)$: = $\frac{d_{i}(b)}{a_{i}(b)} $ black assets fraction by the end of block b.
*   $c_{i}(b)$: amount of clean asset owned by node i by the end of block b
*   $m_{i}(b)$: the amount mined by user i by the end of block b
*   $W(b)$: = {$w_{ij}$}$_{i,j=1,...,N}$ is the adjacency matrix of the transaction network at block b. wij is equal to the amount of bitcoin moved from user i to user j at block b.
*   $N = Nusers = |Nusers|$ total number of users in the network.

Below is a summary of the dynamics of the SIR model: